In [1]:
from IPython import display
import math
from pprint import pprint
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='darkgrid', context='talk', palette='Dark2')
import praw
from praw.models import MoreComments
import spacy
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

In [6]:
nltk.download('vader_lexicon')
nltk.download('punkt')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\tomah\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tomah\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [7]:
spacy.prefer_gpu()
nlp = spacy.load("en_core_web_sm")
sent = "Honestly.. The french just wants it more than other communities. Other streamer communities except for the daliban are just fucking around lol"
doc=nlp(sent)

sub_toks = [tok for tok in doc if (tok.dep_ == "nsubj") ]

print(sub_toks) 

[french, it, communities]


In [8]:
reddit = praw.Reddit(client_id='sRUAAFcF92SAOAcApXtOgA',
                     client_secret='NWHiWc_yM5fCscnAdtDq7HlQKf8U5Q',
                     user_agent='tomtom2352')

In [9]:
def removeWhiteSpace(word):
    return word.strip()

for submission in reddit.subreddit('LivestreamFail').hot(limit=25):
    splitText = submission.link_flair_text.split(":")
    splitText = splitText[len(splitText)-1].split("|")[0].strip()

sia = SIA()
results = []

line = "I don't see dean on my screen this whole convo was hilarious"
pol_score = sia.polarity_scores(line)
pol_score['headline'] = line
results.append(pol_score)

pprint(results[:5], width=100)

sent_text = nltk.sent_tokenize("This is so hilarious I went to see them get into a petty war. After seeing XQC do GTA RP I don't think anybody could be pettier than him")
print(sent_text)


[{'compound': 0.4019,
  'headline': "I don't see dean on my screen this whole convo was hilarious",
  'neg': 0.0,
  'neu': 0.787,
  'pos': 0.213}]
['This is so hilarious I went to see them get into a petty war.', "After seeing XQC do GTA RP I don't think anybody could be pettier than him"]


In [10]:
submission = reddit.submission(id='tt4ar6')
comment = submission.comments[1]
comment.refresh()
replies = comment.replies
print(replies[0].body)

santa deniers 4weird


In [11]:
submission = reddit.submission(id='twanfw')
comments = submission.comments

allComments = comments.list()
for cur in allComments:
    if isinstance(cur, MoreComments):
        continue

In [12]:
titles = []
allComments = []

for submission in reddit.subreddit('LivestreamFail').hot(limit=25):
    splitText = submission.link_flair_text.split(":")
    splitText = splitText[len(splitText)-1].split("|")[0].strip()
    print(splitText)
    
    titles.append(submission.title)
    comments = submission.comments.list()
    for comment in comments[1:]:
        if isinstance(comment, MoreComments):
            continue
        allComments.append(comment.body)
print(allComments)

Gorgc
xQc
StreamerBans
xQc
Lacari
Loud
erobb221
Zoil
Loud
AndyMilonakis
LIRIK
BATTLEGROUNDS
AndyMilonakis
Stormfall33
imaqtpie
Ludwig
96teq96
Loud
xQc
Atabani
BATTLEGROUNDS
summit1g
sennyk4
Velyna
Nmplol
['saw gorgc on my recommended playing slots and it gave me a chuckle LULE', 'minds changes when money talks', 'NotLikeThis Gorgc', 'everyone has their price, LUL', "This is so disappointing. I know Dota is not exactly the most popular game on Twitch but it's not like he's struggling with viewership. Except for Arteezy, he's pretty much the no 1 Dota 2 streamer nowadays. I really liked his vocal opinion against gambling and NFTs. Shame..", 'after he mentioned that he likes to gamble off-stream i was sure this would happen.', '[He said it himself that he "might crack one day"](https://clips.twitch.tv/VenomousWittyWalrusNomNom-bMomXXg8NxXQohyd)', 'Money money money money money money money money moneyyyyyy...', "Reminds me of Asmon ranting about spending money on anything in a game that is

In [13]:
sia = SIA()
results = []
total = 0
positive = 0
negative = 0
neutral = 0

for line in allComments:
    pol_score = sia.polarity_scores(line)
    pol_score['headline'] = line
    results.append(pol_score)
    total += 1
    if pol_score['compound'] >= 0.05:
        positive += 1
    elif pol_score['compound'] <= -0.05:
        negative += 1
    else:
        neutral += 1

pprint(results[:20], width=100)
print(100*positive/total)
print(100*negative/total)
print(100*neutral/total)

[{'compound': 0.6486,
  'headline': 'saw gorgc on my recommended playing slots and it gave me a chuckle LULE',
  'neg': 0.0,
  'neu': 0.613,
  'pos': 0.387},
 {'compound': 0.0,
  'headline': 'minds changes when money talks',
  'neg': 0.0,
  'neu': 1.0,
  'pos': 0.0},
 {'compound': 0.0, 'headline': 'NotLikeThis Gorgc', 'neg': 0.0, 'neu': 1.0, 'pos': 0.0},
 {'compound': 0.0, 'headline': 'everyone has their price, LUL', 'neg': 0.0, 'neu': 1.0, 'pos': 0.0},
 {'compound': 0.7551,
  'headline': 'This is so disappointing. I know Dota is not exactly the most popular game on '
              "Twitch but it's not like he's struggling with viewership. Except for Arteezy, he's "
              'pretty much the no 1 Dota 2 streamer nowadays. I really liked his vocal opinion '
              'against gambling and NFTs. Shame..',
  'neg': 0.137,
  'neu': 0.632,
  'pos': 0.23},
 {'compound': 0.6249,
  'headline': 'after he mentioned that he likes to gamble off-stream i was sure this would happen.',
  'ne

In [14]:
df = pd.DataFrame.from_records(results)
with pd.ExcelWriter('data/learnData.xlsx', mode='a') as writer:  
    df.to_excel(writer, sheet_name='Sheet3')

In [15]:
df.head()

,neg,neu,pos,compound,headline
0,0.000,0.613,0.387,0.6486,saw gorgc on my recommended playing slots and ...
1,0.000,1.000,0.000,0.0000,minds changes when money talks
2,0.000,1.000,0.000,0.0000,NotLikeThis Gorgc
3,0.000,1.000,0.000,0.0000,"everyone has their price, LUL"
4,0.137,0.632,0.230,0.7551,This is so disappointing. I know Dota is not e...
